In [1]:
ip = 'FPV'
num_nbhrs = 100
percent = 0.1
fstring = "_".join([ip,str(percent*100)+"percent",str(num_nbhrs-1)+"nn","mst"])
nodefile = "_".join([fstring,"nodes"])+".csv"
edgefile = "_".join([fstring,"edges"])+".csv"
vectorfile = "_".join([fstring,"vectors"])+".csv"

In [2]:
letters = list("PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKMIGGIGGFIKVRQYDQILIEICGHKAIGTVLVGPTPVNIIGRNLLTQIGCTLNF")

In [3]:
# get the connected component out of the cspanning tree 
f = open(edgefile)
for line in f:
    break
all_edges = []
for line in f:
    e = [int(x) for x in line.strip().split(",")]
    all_edges.append(e)
f.close()

In [4]:
import networkx as nx
def extract_biggest_connected_mst(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    for c in nx.connected_components(G):
        break
    filtered_edges = list(filter(lambda x:x[0] in c and x[1] in c,edges))
    return sorted(c),filtered_edges
nodes, edges = extract_biggest_connected_mst(all_edges)

In [5]:
def is_equal(a,b):
    return all(map(lambda x:x[0]==x[1], zip(a,b)))

In [6]:
# sainity check, the index of vectors vs the index of the sequences
f = open('PI_DataSet_6_19_PI_avg_dist_vec_210.txt')
g = open('PI_DataSet_6_19_expanded.txt')
for line in g:
    header = line.split("\t")
    end = header. index("P1")
    break
count = 0 
for line in g:
    front1 = line.split("\t")[:end]
    fline = next(f)
    front2 = fline.split("\t")[:end]
    assert front1 == front2
    count += 1
f.close()
g.close()

In [7]:
# filter vectors only corresponsing to node_indices. 
node_vectors = []
node_ptr = 0
sainity = []
file_ptr = -1
f = open(vectorfile)
for line in f:
    assert line == '\n'
    print line
    break # account for the blank line
for line in f:
    file_ptr += 1
    if node_ptr < len(nodes) and file_ptr == nodes[node_ptr]:
        node_vectors.append(line)
        # sainity.append(count)
        node_ptr += 1
assert is_equal(sainity, nodes) == True
del sainity
f.close()
assert len(node_vectors) == len(nodes)

In [8]:
# get the header
h = open('PI_DataSet_6_19_expanded.txt')
for line in h:
    header = line.split("\t")
    end = header.index("P1")
    inh_ind = header.index(ip)
    break
h.close()

In [9]:
# we need to patch sequences with vectors. So, first get its inds from the bigger file
g = open('PI_DataSet_6_19_PI_avg_dist_vec_210.txt')
f = enumerate(node_vectors)
seq_indices = []
inh_values = []
count = -1
vec_count = 1
vvec = next(f)[1].strip().split(",")
for line in g:
    line = line.strip().split("\t")
    vec = line[end:]
    count += 1
    if vec[0]==vvec[0] and vec[1]==vvec[1]:
        if is_equal(vec, vvec):
            seq_indices.append(count)
            inh_values.append(float(line[inh_ind]))
            try:
                vvec = next(f)[1].strip().split(",")
                vec_count += 1
            except:
                break
assert len(seq_indices)==vec_count
assert vec_count == len(node_vectors)
assert vec_count == len(nodes)
g.close()

In [10]:
# patch sequences with nodes
h = open('PI_DataSet_6_19_expanded.txt')
for line in h:
    break # for header 
file_ctr = -1
seqs = []
pointer = 0
for line in h:
    file_ctr += 1
    if pointer < len(seq_indices) and file_ctr == seq_indices[pointer]:
        seq = line.strip().split("\t")[end:]
        seqs.append(seq)
        pointer += 1
h.close()
assert len(seqs)==vec_count

In [11]:
# here, we have zip(nodes, node_vectors, seqs, inh_values) as node features and edges on node_inds

In [12]:
def count_difference(a,b):
    return sum([int(x[0]!=x[1]) for x in zip(a,b)])

In [14]:
from collections import defaultdict
difference_index = defaultdict(list)
for i,ind in enumerate(nodes):
    seq = seqs[i]
    diff = count_difference(seq,letters)
    difference_index[diff].append(ind)

In [15]:
root = None
for i in range(30):
    if len(difference_index[i])!=0:
        root = difference_index[i][0]
        break

In [16]:
tree = nx.Graph()
tree.add_edges_from(edges)

In [17]:
assert root in tree.nodes

In [18]:
leaves = [i for  i in nodes if tree.degree(i)==1]

In [19]:
len(leaves)

12697

In [20]:
node_inds = {node:i for i,node in enumerate(nodes)}

In [21]:
# filter the leaf nodes with resistance greater than 3
high_inh_leaves = filter(lambda x:inh_values[node_inds[x]]>=3,leaves)

In [22]:
len(high_inh_leaves)

8062

In [23]:
inh_values[node_inds[root]]

0.5

In [24]:
for ind in difference_index[1]:
    print ind,inh_values[node_inds[ind]]

3 1.0
454 0.6
1335 48.0
1931 48.0
21019 0.7
27958 0.5
31264 0.5


In [25]:
mininh = min(inh_values)
[count_difference(letters,seqs[i]) for i in range(len(node_inds)) if inh_values[i]==mininh]

[5,
 9,
 11,
 10,
 10,
 12,
 9,
 10,
 9,
 10,
 11,
 10,
 9,
 11,
 11,
 11,
 12,
 11,
 12,
 12,
 13,
 13,
 13,
 9]

In [58]:
[n for n in nodes if inh_values[node_inds[n]]==mininh]

[420,
 20340,
 20341,
 20342,
 20343,
 20344,
 20345,
 20346,
 20347,
 20348,
 20349,
 20350,
 20351,
 20352,
 20353,
 20354,
 20355,
 20356,
 20357,
 20358,
 20359,
 20360,
 20361,
 31192]

In [27]:
# get the shortest path from root to high leaves
shortest_paths = []
for i,l in enumerate(high_inh_leaves):
    p = nx.shortest_path(tree,root, l)
    shortest_paths.append(p)
    if i%1000 == 0:
        print i 

0
1000
2000
3000
4000
5000
6000
7000
8000


In [40]:
def get_inh_for_path(p):
    return [inh_values[node_inds[n]] for n in p]
def get_mutation_for_path(p):
    return [mutations[node_inds[n]] for n in p]

In [29]:
shortest_paths_inhs = [get_inh_for_path(p) for p in shortest_paths]

In [39]:
mutations = [count_difference(seq,letters) for seq in seqs]

In [41]:
shortest_paths_mutations = [get_mutation_for_path(p) for p in shortest_paths]

In [34]:
def is_monotone_increasing(s):
    return all(map(lambda x:s[x]<=s[x+1],range(len(s)-1)))

In [42]:
sum(map(lambda s:int(is_monotone_increasing(s)),shortest_paths_mutations))

1

In [38]:
sum(map(lambda s:int(is_monotone_increasing(s)),shortest_paths_inhs))

3

In [ ]:
# What is the correlation between higher mutation and higher inhibitance in general? 

In [43]:
import scipy.stats 

In [44]:
scipy.stats.pearsonr(inh_values,mutations)

(0.17706880905008163, 2.9203149658232873e-281)

In [51]:
import numpy as np
tmp = [node_inds[l] for l in leaves]
scipy.stats.pearsonr(np.asarray(inh_values)[tmp],np.asarray(mutations)[tmp])

(0.16262035409303566, 5.686375812722127e-76)

In [53]:
one_root = 27958 # this guy has a one mutation with inh = 0.5, lowest inh for 

In [54]:
# Shortest paths between root and these leaves
shortest_paths = []
for i,l in enumerate(high_inh_leaves):
    p = nx.shortest_path(tree,one_root, l)
    shortest_paths.append(p)
    if i%1000 == 0:
        print i 

0
1000
2000
3000
4000
5000
6000
7000
8000


In [55]:
shortest_paths_inhs = [get_inh_for_path(p) for p in shortest_paths]
shortest_paths_mutations = [get_mutation_for_path(p) for p in shortest_paths]

In [57]:
print sum(map(lambda s:int(is_monotone_increasing(s)),shortest_paths_mutations))
print sum(map(lambda s:int(is_monotone_increasing(s)),shortest_paths_inhs))

0
3


In [59]:
shortest_paths = []
for i,l in enumerate(high_inh_leaves):
    p = nx.shortest_path(tree,420, l)
    shortest_paths.append(p)
    if i%1000 == 0:
        print i 
shortest_paths_inhs = [get_inh_for_path(p) for p in shortest_paths]
shortest_paths_mutations = [get_mutation_for_path(p) for p in shortest_paths]
print sum(map(lambda s:int(is_monotone_increasing(s)),shortest_paths_mutations))
print sum(map(lambda s:int(is_monotone_increasing(s)),shortest_paths_inhs))

0
1000
2000
3000
4000
5000
6000
7000
8000
1
131


In [ ]:
max_ = max(map(lambda x: len(x[1]), difference_index.items()))

In [ ]:
map(lambda x: x[0], filter(lambda x:len(x[1])==max_,difference_index.items()))

In [ ]:
max(difference_index.keys())

In [ ]:
nx.shortest_path(root,high_inh_leaves[0])

In [ ]:
root in high_inh_leaves

In [ ]:
len(cnodes),len(cedges)